In [32]:
from faker import Faker
import re
import random

fake = Faker()

def take_initial(s):
    initials = [i[0] for i in s.split()]
    while len(initials) < 2:
        initials.append(fake.random_uppercase_letter())
    return ''.join(initials).upper()

def wrap_single_quote(s):
    return "'" + s + "'"

# compartment
FN = 'FN'
AD = 'AD'
OP = 'OP'

# security level
SL_P = 'P'
SL_C = 'C'
SL_S = 'S'
SL_HS = 'HS'

# group 
UNIV = 'UNIV'

# ols column name
ols = 'ols_pol_col'

def get_school_group(school_id):
    return 's_{}'.format(school_id)

def get_course_schedule_group(school_id, cs_id):
    return 's_{}_{}'.format(school_id, cs_id)

def get_security_label(security_level, compartments, groups):
    g = ','.join(groups)
    c = ','.join(compartments)
    return "CHAR_TO_LABEL('ols_pol','{}:{}:{}')".format(security_level, c, g)

In [33]:
school_num = 2
school_security_level = SL_P
school_id = [i+1 for i in range(school_num)]
school_name = ['School of ' + fake.job().split()[0] + ''.join(fake.random_letters(length=3)) for i in range(school_num)]
school_email = [ take_initial(school_name[i]) + str(i) + '@nus.edu.sg' for i in range(school_num)]
school_address = [school_name[i] + ', ' + fake.address().replace('\n', '') for i in range(school_num)]
school_postal = [fake.postalcode() for i in range(school_num)]
school_tel = [fake.phone_number() for i in range(school_num)]
school_label = ['SOC', 'BIZ']

school_insert_sql = 'INSERT ALL\n'+ '\n'.join(
    ['INTO school (id, name, email, address, postal, tel)\
    VALUES ({},{},{},{},{},{})'.format(
    school_id[i],
    wrap_single_quote(school_name[i]),
    wrap_single_quote(school_email[i]),
    wrap_single_quote(school_address[i]),
    wrap_single_quote(school_postal[i]),
    wrap_single_quote(school_tel[i])) for i in range(school_num)]) + '\n SELECT * FROM dual;\n\n'

# print(school_insert_sql)

In [34]:
academic_num = 4
asy_slevel = SL_P
asy_id = [i+1 for i in range(academic_num)]
academic_start_year = [str(2017-academic_num//2+i) for i in range(academic_num)]
academic_end_year = [str(2018-academic_num//2+i) for i in range(academic_num)]
asy_sl = [get_security_label(asy_slevel, [],[]) for i in asy_id]

academic_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ["INTO academic_year (id, start_year, end_year)\
    VALUES ({},{},{})".format(
    asy_id[i],
    wrap_single_quote(academic_start_year[i]),
    wrap_single_quote(academic_end_year[i])) for i in range(academic_num)]) + '\n SELECT * FROM dual;\n\n'

# print(academic_insert_sql)

In [35]:
semster_num = academic_num * 2
semster_start_end_dates = [('-08-01 00:00:00 +08:00','-12-31 00:00:00 +08:00'), ('-01-01 00:00:00 +08:00','-05-31 00:00:00 +08:00')]
sem_id = [i+1 for  i in range(semster_num)]
sem_slevel = SL_P
semster = [i%2+1 for i in range(semster_num)]
semster_start_time = [str(2017-semster_num//4+(i+1)//2)+semster_start_end_dates[i%2][0] for i in range(semster_num)]
semster_end_time = [str(2017-semster_num//4+(i+1)//2)+semster_start_end_dates[i%2][1] for i in range(semster_num)]
sem_sl = [get_security_label(sem_slevel, [],[]) for i in range(semster_num)]
semster_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO semester (id, semester, start_date, end_date, academic_year_id)\
    VALUES({}, {}, TIMESTAMP {},TIMESTAMP {}, {})'.format(
    sem_id[i],
    str(semster[i]),
    wrap_single_quote(semster_start_time[i]),
    wrap_single_quote(semster_end_time[i]),
    str(i//2+1)) for i in range(semster_num)]) + '\n SELECT * FROM dual;\n\n'

# print(semster_insert_sql)

In [36]:
admin_officer_num = 2 * school_num

admin_officer_id = [ fake.uuid4() for i in range(admin_officer_num)]
admin_officer_name = [ fake.name() for i in range(admin_officer_num)]
admin_officer_office_email = [ admin_officer_name[i].replace(" ", ".") + str(i) + '@nus.edu.sg' for i in range(admin_officer_num)]
admin_officer_office_address = [ fake.address().replace('\n', '') for i in range(admin_officer_num)]
admin_officer_office_postal = [ fake.postalcode() for i in range(admin_officer_num)]
admin_officer_office_tel = [ fake.phone_number() for i in range(admin_officer_num)]
admin_officer_school_id = [ i%school_num+1 for i in range(admin_officer_num)]

admin_officer_insert_sql = 'INSERT ALL\n' + '\n'.join(['INTO admin_officer (id, first_name, last_name, gender, email, office_address, office_postal, office_tel, school_id) VALUES({},{},{},{},{},{},{},{},{})'.format(
    wrap_single_quote(admin_officer_id[i]),
    wrap_single_quote(admin_officer_name[i].split(' ')[0]),
    wrap_single_quote(admin_officer_name[i].split(' ')[1]),
    wrap_single_quote(random.choice(['F', 'M'])),
    wrap_single_quote(admin_officer_office_email[i]),
    wrap_single_quote(admin_officer_office_address[i]),
    wrap_single_quote(admin_officer_office_postal[i]),
    wrap_single_quote(admin_officer_office_tel[i]),
    str(admin_officer_school_id[i])) for i in range(admin_officer_num)]) + '\n SELECT * FROM dual;\n\n'

In [37]:
professor_num = 2 * school_num
professor_id = [ fake.uuid4() for i in range(professor_num)]
professor_name = [ fake.name() for i in range(professor_num)]
professor_office_email = [ professor_name[i].replace(" ", ".") + str(i) + '@nus.edu.sg' for i in range(professor_num)]
professor_office_address = [ fake.address().replace('\n', '') for i in range(professor_num)]
professor_office_postal = [ fake.postalcode() for i in range(professor_num)]
professor_office_tel = [ fake.phone_number() for i in range(professor_num)]
professor_school_id = [ i%school_num+1 for i in range(professor_num)]

professor_insert_sql = 'INSERT ALL\n' + '\n'.join(['INTO professor (id, first_name, last_name, gender, email, office_address, office_postal, office_tel, school_id) VALUES({},{},{},{},{},{},{},{},{})'.format(
    wrap_single_quote(professor_id[i]),
    wrap_single_quote(professor_name[i].split(' ')[0]),
    wrap_single_quote(professor_name[i].split(' ')[1]),
    wrap_single_quote(random.choice(['F', 'M'])),
    wrap_single_quote(professor_office_email[i]),
    wrap_single_quote(professor_office_address[i]),
    wrap_single_quote(professor_office_postal[i]),
    wrap_single_quote(professor_office_tel[i]),
    str(professor_school_id[i])) for i in range(professor_num)]) + '\n SELECT * FROM dual;\n\n'

In [38]:
student_num = 2 * school_num
stu_slevel = SL_C
student_id = [ fake.uuid4() for i in range(student_num)]
student_name = [ fake.name() for i in range(student_num)]
student_email = [ student_name[i].replace(" ", ".") + str(i) + '@nus.edu.sg' for i in range(student_num)]
student_address = [ fake.address().replace('\n', '') for i in range(student_num)]
student_postal = [ fake.postalcode() for i in range(student_num)]
student_tel = [ fake.phone_number() for i in range(student_num)]
stu_sch_id = [ i%school_num+1 for i in range(student_num)]

In [39]:
loan_num = student_num // 2
loan_slevel = SL_S
loan_id = [i+1 for i in range(loan_num)]
loan_amount = [ random.randint(10000,50000) for i in range(loan_num)]
loan_start_date = [ fake.past_date().replace().strftime("%Y-%m-%d %H:%M:%S") + ' +08:00' for i in range(loan_num)]
loan_due_date = [ fake.future_date(end_date='+365d').strftime("%Y-%m-%d %H:%M:%S") + ' +08:00' for i in range(loan_num)]
loan_amount_paid = [ "{0:.2f}".format(random.random() * loan_amount[i]) for i in range(loan_num)]
loan_student_id = [ student_id[i*2] for i in range(loan_num)]
loan_sl = [get_security_label(loan_slevel, [FN], [school_label[i%2]]) for i in range(loan_num)]

loan_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO loan (id, amount, start_date, due_date, paid_amount, student_id, {})\
    VALUES({},{},TIMESTAMP {},TIMESTAMP {},{},{},{})'.format(
    ols,
    loan_id[i],
    str(loan_amount[i]),
    wrap_single_quote(loan_start_date[i]),
    wrap_single_quote(loan_due_date[i]),
    str(loan_amount_paid[i]),
    wrap_single_quote(loan_student_id[i]),
    loan_sl[i]) for i in range(loan_num)]) + '\n SELECT * FROM dual;\n\n'

# print(loan_insert_sql)

In [40]:
course_id = ['CS5228', 'BT4211']
course_num = len(course_id)
course_index = [ fake.random_number(digits=4) for i in range(course_num)]
course_name = [ fake.job()[:50] for i in range(course_num)]
course_description = [ fake.sentence() for i in range(course_num)]
course_school_id = [1,2]
course_label = course_id[:]
course_insert_sql = 'INSERT ALL\n' + '\n'.join(['INTO course (id, idx, name, description, school_id) VALUES({},{},{},{},{})'.format(
    i+1,
    str(course_index[i]),
    wrap_single_quote(course_name[i]),
    wrap_single_quote(course_description[i]),
    str(course_school_id[i])) for i in range(course_num)]) + '\n SELECT * FROM dual;\n\n'

print(course_insert_sql)

INSERT ALL
INTO course (id, idx, name, description, school_id) VALUES(1,4463,'Operations geologist','If create green design student place part.',1)
INTO course (id, idx, name, description, school_id) VALUES(2,7512,'Scientist, research (life sciences)','Himself quite smile old computer.',2)
 SELECT * FROM dual;




In [41]:
course_schedule_num = course_num
cs_id = [i+1 for i in range(course_schedule_num)]
course_schedule_course_id = [ i%course_num+1 for i in range(course_schedule_num)] 
course_schedule_semster_id = [ i%semster_num+1 for i in range(course_schedule_num)]
course_schedule_professor_id = [ professor_id[i%professor_num] for i in range(course_schedule_num)]

course_schedule_insert_sql = 'INSERT ALL\n' + '\n'.join(['INTO course_schedule (id, course_id, semester_id, professor_id) VALUES({},{},{},{})'.format(
    i+1,
    str(course_schedule_course_id[i]),
    str(course_schedule_semster_id[i]),
    wrap_single_quote(course_schedule_professor_id[i])) for i in range(course_schedule_num)]) + '\n SELECT * FROM dual;\n\n'

print(course_insert_sql)

INSERT ALL
INTO course (id, idx, name, description, school_id) VALUES(1,4463,'Operations geologist','If create green design student place part.',1)
INTO course (id, idx, name, description, school_id) VALUES(2,7512,'Scientist, research (life sciences)','Himself quite smile old computer.',2)
 SELECT * FROM dual;




In [42]:
course_enrollment_num = student_num
ce_slevel = SL_S
ce_id = [i+1 for i in range(course_enrollment_num)]
ce_stu_id = [ student_id[i] for i in range(course_enrollment_num)]
ce_cs_id = [ cs_id[i//3] for i in range(course_enrollment_num)]
ce_sl = [get_security_label(ce_slevel, [OP], [course_label[i//2]]) for i in range(course_enrollment_num)]

course_enrollment_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO course_enrollment(id, student_id,course_schedule_id,{})\
    VALUES({},{},{},{})'.format(
    ols,
    str(ce_id[i]),
    wrap_single_quote(ce_stu_id[i]),
    str(ce_cs_id[i]),
    ce_sl[i]) for i in range(course_enrollment_num)]) + '\n SELECT * FROM dual;\n\n'

print(course_enrollment_insert_sql)

INSERT ALL
INTO course_enrollment(id, student_id,course_schedule_id,ols_pol_col)    VALUES(1,'208843ec-ae50-8d50-389e-8f88ac7232cd',1,CHAR_TO_LABEL('ols_pol','S:OP:CS5228'))
INTO course_enrollment(id, student_id,course_schedule_id,ols_pol_col)    VALUES(2,'08795cf9-ac6d-00f0-9ab1-7d7bb244cdb2',1,CHAR_TO_LABEL('ols_pol','S:OP:CS5228'))
INTO course_enrollment(id, student_id,course_schedule_id,ols_pol_col)    VALUES(3,'f262d8a9-59e5-0c53-d2db-311df56527df',1,CHAR_TO_LABEL('ols_pol','S:OP:BT4211'))
INTO course_enrollment(id, student_id,course_schedule_id,ols_pol_col)    VALUES(4,'dccf533f-2972-0e80-dd55-8462989c8216',2,CHAR_TO_LABEL('ols_pol','S:OP:BT4211'))
 SELECT * FROM dual;




In [43]:
stu_group_label = [get_security_label(stu_slevel, [], [course_label[i//3]]) for i in range(student_num)]

student_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO student (id, first_name, last_name, gender, matric_no, email, address, postal, tel, school_id, {}) \
    VALUES({},{},{},{},{},{},{},{},{},{},{})'.format(
    ols,
    wrap_single_quote(student_id[i]),
    wrap_single_quote(student_name[i].split(' ')[0]),
    wrap_single_quote(student_name[i].split(' ')[1]),
    wrap_single_quote(random.choice(['F', 'M'])),
    wrap_single_quote('AW' + student_id[i][:18]),
    wrap_single_quote(student_email[i]),
    wrap_single_quote(student_address[i]),
    wrap_single_quote(student_postal[i]),
    wrap_single_quote(student_tel[i]),
    str(stu_sch_id[i]), 
    stu_group_label[i]) for i in range(student_num)]) + '\n SELECT * FROM dual;\n\n'

print(student_insert_sql)

INSERT ALL
INTO student (id, first_name, last_name, gender, matric_no, email, address, postal, tel, school_id, ols_pol_col)     VALUES('208843ec-ae50-8d50-389e-8f88ac7232cd','Jeffrey','Durham','F','AW208843ec-ae50-8d50','Jeffrey.Durham0@nus.edu.sg','15966 Sheppard Drive Suite 063Johnbury, GA 73407','56559','335-472-9235x903',1,CHAR_TO_LABEL('ols_pol','C::CS5228'))
INTO student (id, first_name, last_name, gender, matric_no, email, address, postal, tel, school_id, ols_pol_col)     VALUES('08795cf9-ac6d-00f0-9ab1-7d7bb244cdb2','Robert','Molina','M','AW08795cf9-ac6d-00f0','Robert.Molina1@nus.edu.sg','41621 Michael Port Apt. 852Zacharyberg, AR 81072','67317','191.284.4171x4324',2,CHAR_TO_LABEL('ols_pol','C::CS5228'))
INTO student (id, first_name, last_name, gender, matric_no, email, address, postal, tel, school_id, ols_pol_col)     VALUES('f262d8a9-59e5-0c53-d2db-311df56527df','Greg','Russo','F','AWf262d8a9-59e5-0c53','Greg.Russo2@nus.edu.sg','6089 Nelson Knolls Suite 928Blackburnstad, NV 4

In [44]:
grade_num = course_enrollment_num
grade_slevel = SL_S
grade_ce_id = ce_id[:]
grades = [fake.random_number(digits=2) for i in range(grade_num)]
grades_sl = [get_security_label(ce_slevel, [AD], [course_label[i//2]]) for i in range(grade_num)]

grade_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO grade(course_enrollment_id, grade, {})\
    VALUES({},{},{})'.format(
    ols,
    str(grade_ce_id[i]),
    str(grades[i]),
    grades_sl[i]) for i in range(course_enrollment_num)]) + '\n SELECT * FROM dual;\n\n'

In [45]:
print(grade_insert_sql)

INSERT ALL
INTO grade(course_enrollment_id, grade, ols_pol_col)    VALUES(1,77,CHAR_TO_LABEL('ols_pol','S:AD:CS5228'))
INTO grade(course_enrollment_id, grade, ols_pol_col)    VALUES(2,0,CHAR_TO_LABEL('ols_pol','S:AD:CS5228'))
INTO grade(course_enrollment_id, grade, ols_pol_col)    VALUES(3,45,CHAR_TO_LABEL('ols_pol','S:AD:BT4211'))
INTO grade(course_enrollment_id, grade, ols_pol_col)    VALUES(4,46,CHAR_TO_LABEL('ols_pol','S:AD:BT4211'))
 SELECT * FROM dual;




In [46]:
school_fee_num = student_num
sf_slevel = SL_S
school_fee_student_id = [ student_id[i] for i in range(school_fee_num)]
school_fee_semster_id = [ i%semster_num+1 for i in range(school_fee_num)]
school_fee_school_fee = [ fake.random_number(digits=5) for i in range(school_fee_num)]
school_fee_school_fee_paid = [ "{0:.2f}".format(random.random()*school_fee_school_fee[i]) for i in range(school_fee_num)]
sf_sl = [get_security_label(sf_slevel, [FN], [school_label[i//2]]) for i in range(school_fee_num)]

school_fee_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO school_fee(student_id, semester_id, school_fee, school_fee_paid, {})\
    VALUES({},{},{},{},{})'.format(
    ols,
    wrap_single_quote(school_fee_student_id[i]),
    str(school_fee_semster_id[i]),
    str(school_fee_school_fee[i]),
    str(school_fee_school_fee_paid[i]),
    sf_sl[i]) for i in range(school_fee_num)]) + '\n SELECT * FROM dual;\n\n'

print(school_fee_insert_sql)

INSERT ALL
INTO school_fee(id, student_id,semester_id,school_fee,school_fee_paid, ols_pol_col)    VALUES('208843ec-ae50-8d50-389e-8f88ac7232cd',1,50388,49800.47,CHAR_TO_LABEL('ols_pol','S:FN:SOC'))
INTO school_fee(id, student_id,semester_id,school_fee,school_fee_paid, ols_pol_col)    VALUES('08795cf9-ac6d-00f0-9ab1-7d7bb244cdb2',2,51328,5417.89,CHAR_TO_LABEL('ols_pol','S:FN:SOC'))
INTO school_fee(id, student_id,semester_id,school_fee,school_fee_paid, ols_pol_col)    VALUES('f262d8a9-59e5-0c53-d2db-311df56527df',3,94795,57152.55,CHAR_TO_LABEL('ols_pol','S:FN:BIZ'))
INTO school_fee(id, student_id,semester_id,school_fee,school_fee_paid, ols_pol_col)    VALUES('dccf533f-2972-0e80-dd55-8462989c8216',4,92442,79478.66,CHAR_TO_LABEL('ols_pol','S:FN:BIZ'))
 SELECT * FROM dual;




In [47]:
staff_num = professor_num + admin_officer_num
staff_slevel = SL_HS
staff_id = professor_id + admin_officer_id
salary = [fake.random_number(digits=5) for i in range(staff_num)]
staff_sl = [get_security_label(staff_slevel,[FN], []) for i in range(staff_num)]
staff_insert_sql = 'INSERT ALL\n' + '\n'.join(
    ['INTO staff_salary(staff_id, salary, {})\
    VALUES({},{},{})'.format(
    ols,
    wrap_single_quote(staff_id[i]),
    str(salary[i]),
    staff_sl[i]) for i in range(staff_num)]) + '\n SELECT * FROM dual;\n\n'
print(staff_insert_sql)

INSERT ALL
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('cbb24338-3103-8fc2-a704-5c27bf1ea0c3',65246,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('bbc2c618-b619-bb2e-591f-d18d7e4322dd',22906,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('a9fe72ea-1c93-745d-48c2-0b07f20abbb8',93302,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('73291bd2-6414-56a6-5f00-5deae7abeb9b',68554,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('b81c94e9-00c7-190c-3082-3e0d8ee4ebd0',64828,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('6134c9c0-1c6a-a2b3-4f43-b28246328b59',35448,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary(staff_id, salary, ols_pol_col)    VALUES('20496c78-4e1f-dd97-8e73-eec260f4a0e2',7959,CHAR_TO_LABEL('ols_pol','HS:FN:'))
INTO staff_salary

In [48]:
with open('insert_data.sql', 'w') as f:
    f.write(school_insert_sql)

In [49]:
with open('insert_data.sql', 'a') as f:
    f.write(academic_insert_sql)

In [50]:
with open('insert_data.sql', 'a') as f:
    f.write(semster_insert_sql)

In [51]:
with open('insert_data.sql', 'a') as f:
    f.write(admin_officer_insert_sql)

In [52]:
with open('insert_data.sql', 'a') as f:
    f.write(professor_insert_sql)

In [53]:
with open('insert_data.sql', 'a') as f:
    f.write(course_insert_sql)

In [54]:
with open('insert_data.sql', 'a') as f:
    f.write(course_schedule_insert_sql)

In [55]:
with open('insert_data.sql', 'a') as f:
    f.write(student_insert_sql)

In [56]:
with open('insert_data.sql', 'a') as f:
    f.write(course_enrollment_insert_sql)

In [57]:
with open('insert_data.sql', 'a') as f:
    f.write(school_fee_insert_sql)

In [58]:
with open('insert_data.sql', 'a') as f:
    f.write(loan_insert_sql)

In [59]:
with open('insert_data.sql', 'a') as f:
    f.write(grade_insert_sql)

In [60]:
with open('insert_data.sql', 'a') as f:
    f.write(staff_insert_sql)